# 데이터 전처리

#### 머신러닝 코드 구조 <br>

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")



## 00.환경준비

### 01.Import

In [ ]:
# 라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 02.data loading


* 주가 데이터 가져오기

In [ ]:
stock = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/SK.csv') 
stock.drop('AdjClose', axis=1, inplace=True)

exch_rate = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/USD_KRW.csv')
exch_rate.drop(['open', 'high', 'low'], axis=1, inplace=True)
exch_rate.rename(columns={'date':'Date', 'close':'exch_Close', 'diff':'exch_Diff'},inplace=True)

In [ ]:
stock.head()

In [ ]:
exch_rate.head()

* 하나의 데이터로 결합

In [ ]:
data = pd.merge(stock, exch_rate, how='left', on='Date')

data.head()

## 10.데이터 이해

### 11.둘러보기

In [ ]:
# 상/하위 몇개 행을 살펴 봅시다.

data.head()

In [ ]:
# 각 칼럼의 타입을 살펴 봅시다.

data.dtypes

In [ ]:
# 행, 열의 개수를 살펴 봅시다.

data.shape

### 12.기초통계량

In [ ]:
# 숫자형 변수들의 통계량을 살펴 봅니다.

data.describe()

### 13.탐색하기

## 20.데이터 준비


### 21.변수 정리

### 22.NaN 처리

In [ ]:
# NaN 확인
data.isnull().sum()

In [ ]:
data['Open'].isnull()

In [ ]:
# NaN 행들을 확인해 봅시다.
data.loc[data['Open'].isnull()]

In [ ]:
data.loc[(data['Date']>='2019-11-12')&(data['Date']<='2019-11-20')]
# 2019-11-14 는 목요일. 원인은 모르나 NaN으로 채워져 있음.

In [ ]:
# 어떻게 조치하는게 좋을까요?
# 방법1 : 행을 제거한다.

data1 = data.dropna(axis=0)
data1.isnull().sum()

In [ ]:
data1.loc[(data1['Date']>='2019-11-12')&(data1['Date']<='2019-11-20')]

In [ ]:
# 방법2 : 이전 값으로 채운다. 

data2 = data.fillna(method='ffill')
data2.isnull().sum()

In [ ]:
data2.loc[(data2['Date']>='2019-11-12')&(data2['Date']<='2019-11-20')]

In [ ]:
# 방법3 : 앞뒤값의 중간값으로 채우기

data3 = data.interpolate(method='linear')
data3.isnull().sum()

In [ ]:
data3.loc[(data3['Date']>='2019-11-12')&(data3['Date']<='2019-11-20')]

#### 실습 9 : NaN 조치

* Q1. 주가 데이터에서 NaN이 변수별로 몇 건씩 있는지 조사하시오.

* Q2. NaN 행을 제거합니다. 제거해서 별도 데이터프레임으로 저장.

* Q3. NaN 행에 이후 행의 값으로 채우고, 별도 데이터프레임으로 저장.
    * method='bfill'

* Q4. NaN 행에 값을 0으로 채우고, 별도 데이터프레임으로 저장.
    * .fillna(0)

* Q5. interpolate를 사용해보시오.
    * .interpolate(method='linear')

### 23.Feature Engineering

* 내일의 주가에 영향을 주는 요인은 무엇을까요?

In [ ]:
data2.head()

#### 1)날짜 데이터 다루기

* 날짜 변수 만들기  
    - 날짜변수를 만드는 이유!  
        - 날짜로부터 추가 변수를 도출해내기 위해서 날짜 변수를 만듭니다.  
        - 만약 추가변수 도출이 끝나면, 날짜변수는 제거합니다.

In [ ]:
data2.dtypes

In [ ]:
# 문자열 형식을 날짜 형식으로 변환
data2['Date'] = pd.to_datetime(data2['Date'])
print(data2.dtypes)

* data2.Date.dt.dayofweek  : The day of the week with Monday=0, Sunday=6.
* data2.Date.dt.day_name() : 요일 이름
* data2.Date.dt.week : 1~53주
* data2.Date.dt.year
* data2.Date.dt.month

* 날짜로 부터 날짜요소 추출하기

In [ ]:
# 요일을 추가해 봅시다. 
data2['WeekDay'] = data2.Date.dt.dayofweek
data2.head()

In [ ]:
# 요일 이름으로 다시 추가.
data2['WeekDay'] = data2.Date.dt.day_name()
data2.head()

#### 2)이전 데이터 붙이기.  
- .shift( )   : 예) 전날 주가, 전날 환율
- .rolling( ) : 예)7일이동평균 주가

In [ ]:
# 전날 주가를 추가합니다.
data2['Close_lag1'] = data2['Close'].shift() # default = 1
data2.head()

In [ ]:
# 7일 이동평균 주가
data2['Close_MA7_lag1'] = data2['Close'].rolling(7, min_periods=1).mean().shift()
data2.head()

#### 실습 10 : feature engineering

* data2 변수를 이용하여 문제를 푸시오

* Q1. 월 데이터를 추가해보시오

* Q2. 전날 거래량 컬럼, 전날 환율 증감 컬럼을 만드시오.

* Q3. "전날 주가 - 전전날 주가" 컬럼을 만드시오.

* Q4. "전날 종가 - 전날 시가" 컬럼을 만드시오.

* Q5. 그 외 추가하고 싶은 변수를 1 ~ 3개 정도 더 추가해보세요.

### 24.Dummy Variable
* 범주형 변수를 숫자로 만드는 방법
* pd.get_dummies, pd.concat, (Pandas Dataframe).drop
* 불필요한 칼럼들 제거


In [ ]:
# dummy variable

dumm_weekday = pd.get_dummies(data2['WeekDay'], drop_first=True)

In [ ]:
dumm_weekday.head()

In [ ]:
data3 = pd.concat([data2, dumm_weekday], axis=1)
data3.head()

* 불필요한 변수를 제거합시다.

* 어떤 변수가 불필요한가요?  
    - 날짜
    - 전날로 뺀 변수의 원본
    - 가변수화 한 변수의 원본
    - 의미 없는 변수

In [ ]:
# 칼럼삭제
drop_x = ['Date','Open','High','Low','Volume','exch_Close','exch_Diff','WeekDay']
data3.drop(drop_x, axis = 1, inplace = True)
data3.head()

In [ ]:
# shift를 하다보내 행에 NaN이 다시 들어갔습니다. dropna로 제거합시다.
data3 = data3.dropna(axis = 0)
data3.head()

#### 실습 11 : dummy variable

* Q1. 주가 데이터에서 월에 대하여 가변수화 합니다.

* Q2. 가변수한 데이터를 기존 데이터와 결합합니다. ( concat )

* Q3. 가변수화 하기 이전의 월 컬럼을 제거하세요.

##### 간결한 방식

In [ ]:
data3 = pd.get_dummies(data3, columns=['Month'], drop_first=True)

In [ ]:
data3.columns

In [ ]:
data3.head()

--------------------------------
여기서 부터는 그냥 실행 해 볼 겁니다.  
데이터를 얼추(!) 준비했으니, 모델링까지 수행해 봅니다.

### 25.Data Split

sklearn의 데이터 분할 함수를 사용해 봅시다.

* 요인, x, feature, 조작변수, 통제변수, 리스크백터, Input, (독립변수)
* 결과, y, target, label, Output, (종속변수)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# features와 target 분리
X = data3.drop('Close', axis=1)
y = data3.iloc[:, 0]

In [ ]:
# 전체에서 train : test = 7 : 3
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
train_x.shape, train_y.shape

### 26.Scaling features




In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 사용할 함수 선언
scaler = MinMaxScaler()

In [ ]:
# 함수 만들고
scaler.fit(train_x)

# 변환시키기
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)


### 27.Dataframe to Numpy array

## 30.모델링

### 31.import

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression        # Linear Model
from sklearn.metrics   import mean_squared_error         # Metric  MSE

### 32.모델선언

In [ ]:
# 모델 선언
model = LinearRegression()

### 33.모델링(학습)

In [ ]:
model.fit( train_x , train_y )

### 34.예측

In [ ]:
# 예측값을 뽑자.
test_pred = model.predict(test_x)

### 35.평가

In [ ]:
# test set에서의 성능 확인
mean_squared_error( test_y, test_pred, squared=False )

나온 결과의 의미는?